In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.name import clean_name
from lib.date import to_datetime_series
from lib.columns import rearrange_personel_columns

In [2]:
df1 = pd.read_excel("../data/New Orleans Civil Service Department/Administrative Data/PPRR_09-15-2020.xlsx")
df1 = df1[[name for name in df1.columns if not name.startswith("Unnamed:")]]
df1.rename(columns={name: name.title() for name in df1.columns}, inplace=True)
df1

,Orgn Desc,Last Name,First Name,Titl E Code,Title Desc,Term Date,Pay Prog Start Date,Annual Salary
0,FISCAL MANAGEMENT,ABBOTT,CHRISTOPHER,C7134,POLICE OFFICER 4,NaT,1992-09-02,46885.0056
1,TRAFFIC,ABBOTT,DAVID,C7134,POLICE OFFICER 4,NaT,1987-01-11,48064.4565
2,POLICE RECRUITS,ABRAM,SHELTON,C7109,POLICE RECRUIT ACADEMY,NaT,2009-06-09,34797.9137
3,FISCAL MANAGEMENT,ADAMS,JAMES,C7133,POLICE OFFICER 3,2009-08-30,1987-06-09,45734.4971
4,PUBLIC INTEGRITY,ADAMS,BRUCE,U0018,ASSISTANT SUPT OF POLICE,NaT,1975-10-16,106440.2966
...,...,...,...,...,...,...,...,...
1976,POLICE RECRUITS,YOUNT,JEFFREY,NaN,POLICE RECRUIT ACADEMY,NaT,NaT,34797.9137
1977,TECHNICIAL SERVICES BUREA,ZALDIVAR,SARA,NaN,POLICE COMPLAINT OPERATOR,NaT,NaT,25508.0588
1978,SIXTH DISTRICT,ZAPICO,DANIEL,NaN,POLICE OFFICER 1,NaT,NaT,36570.7051
1979,THIRD DISTRICT,ZOLLER,RON,NaN,POLICE OFFICER 4,NaT,NaT,44612.2209


In [3]:
df2 = df1[["Orgn Desc", "First Name", "Last Name", "Titl E Code", "Title Desc", "Term Date", "Pay Prog Start Date", "Annual Salary"]]
df2.rename(columns={
    "Term Date": "Termination Date",
    "Orgn Desc": "Department",
    "Titl E Code": "Rank Number #",
    "Title Desc": "Rank",
}, inplace=True)
df2 = rearrange_personel_columns(df2)
for col in ["Last Name", "First Name"]:
    df2.loc[:, col] = df2[col].map(clean_name, na_action="ignore")
for col in ["Termination Date", "Pay Prog Start Date"]:
    df2.loc[:, col] = to_datetime_series(df2[col])
df2

,Last Name,First Name,Department,Rank Number #,Rank,Termination Date,Pay Prog Start Date,Annual Salary
0,Abbott,Christopher,FISCAL MANAGEMENT,C7134,POLICE OFFICER 4,NaT,1992-09-02,46885.0056
1,Abbott,David,TRAFFIC,C7134,POLICE OFFICER 4,NaT,1987-01-11,48064.4565
2,Abram,Shelton,POLICE RECRUITS,C7109,POLICE RECRUIT ACADEMY,NaT,2009-06-09,34797.9137
3,Adams,James,FISCAL MANAGEMENT,C7133,POLICE OFFICER 3,2009-08-30,1987-06-09,45734.4971
4,Adams,Bruce,PUBLIC INTEGRITY,U0018,ASSISTANT SUPT OF POLICE,NaT,1975-10-16,106440.2966
...,...,...,...,...,...,...,...,...
1976,Yount,Jeffrey,POLICE RECRUITS,NaN,POLICE RECRUIT ACADEMY,NaT,NaT,34797.9137
1977,Zaldivar,Sara,TECHNICIAL SERVICES BUREA,NaN,POLICE COMPLAINT OPERATOR,NaT,NaT,25508.0588
1978,Zapico,Daniel,SIXTH DISTRICT,NaN,POLICE OFFICER 1,NaT,NaT,36570.7051
1979,Zoller,Ron,THIRD DISTRICT,NaN,POLICE OFFICER 4,NaT,NaT,44612.2209


In [4]:
df2.to_csv("../output/new-orleans-csd_personel_09-15-2020.csv", index=False)